In [1]:
a = 5
b = 4
c = a + b
print(c)



9


text explaining what code does

#comments
###Long multiple lines of comments###

In [1]:
seconds_in_a_day = 24 * 60 * 60
seconds_in_a_day

86400

In [2]:
seconds_in_a_week = 7 * seconds_in_a_day
seconds_in_a_week

604800

In [6]:
from __future__ import print_function

import tensorflow as tf

# Create a graph.
g = tf.Graph()

# Establish our graph as the "default" graph.
with g.as_default():
  # Assemble a graph consisting of three operations. 
  # (Creating a tensor is an operation.)
  x = tf.constant(8, name="x_const")
  y = tf.constant(5, name="y_const")
  my_sum = tf.add(x, y, name="x_y_sum")
  
  # Task 1: Define a third scalar integer constant z.
  z = tf.constant(4, name="z_const")
  # Task 2: Add z to `my_sum` to yield a new sum.
  new_sum = tf.add(my_sum, z, name="x_y_z_sum")

  # Now create a session.
  # The session will run the default graph.
  with tf.Session() as sess:
    # Task 3: Ensure the program yields the correct grand total.
    print(new_sum.eval())

ModuleNotFoundError: No module named 'tensorflow'

In [10]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

california_housing_dataframe = pd.read_csv("https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv", sep=",")

california_housing_dataframe = california_housing_dataframe.reindex(
    np.random.permutation(california_housing_dataframe.index))
california_housing_dataframe["median_house_value"] /= 1000.0
california_housing_dataframe

california_housing_dataframe.describe()

# Define the input feature: total_rooms.
my_feature = california_housing_dataframe[["total_rooms"]]

# Configure a numeric feature column for total_rooms.
feature_columns = [tf.feature_column.numeric_column("total_rooms")]

# Define the label.
targets = california_housing_dataframe["median_house_value"]

# Use gradient descent as the optimizer for training the model.
my_optimizer=tf.train.GradientDescentOptimizer(learning_rate=0.0000001)
my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)

# Configure the linear regression model with our feature columns and optimizer.
# Set a learning rate of 0.0000001 for Gradient Descent.
linear_regressor = tf.estimator.LinearRegressor(
    feature_columns=feature_columns,
    optimizer=my_optimizer
)

def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
    """Trains a linear regression model of one feature.
  
    Args:
      features: pandas DataFrame of features
      targets: pandas DataFrame of targets
      batch_size: Size of batches to be passed to the model
      shuffle: True or False. Whether to shuffle the data.
      num_epochs: Number of epochs for which data should be repeated. None = repeat indefinitely
    Returns:
      Tuple of (features, labels) for next data batch
    """
  
    # Convert pandas data into a dict of np arrays.
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    # Construct a dataset, and configure batching/repeating.
    ds = Dataset.from_tensor_slices((features,targets)) # warning: 2GB limit
    ds = ds.batch(batch_size).repeat(num_epochs)
    
    # Shuffle the data, if specified.
    if shuffle:
      ds = ds.shuffle(buffer_size=10000)
    
    # Return the next batch of data.
    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

_ = linear_regressor.train(
    input_fn = lambda:my_input_fn(my_feature, targets),
    steps=100
)

# Create an input function for predictions.
# Note: Since we're making just one prediction for each example, we don't 
# need to repeat or shuffle the data here.
prediction_input_fn =lambda: my_input_fn(my_feature, targets, num_epochs=1, shuffle=False)

# Call predict() on the linear_regressor to make predictions.
predictions = linear_regressor.predict(input_fn=prediction_input_fn)

# Format predictions as a NumPy array, so we can calculate error metrics.
predictions = np.array([item['predictions'][0] for item in predictions])

# Print Mean Squared Error and Root Mean Squared Error.
mean_squared_error = metrics.mean_squared_error(predictions, targets)
root_mean_squared_error = math.sqrt(mean_squared_error)
print("Mean Squared Error (on training data): %0.3f" % mean_squared_error)
print("Root Mean Squared Error (on training data): %0.3f" % root_mean_squared_error)

min_house_value = california_housing_dataframe["median_house_value"].min()
max_house_value = california_housing_dataframe["median_house_value"].max()
min_max_difference = max_house_value - min_house_value

print("Min. Median House Value: %0.3f" % min_house_value)
print("Max. Median House Value: %0.3f" % max_house_value)
print("Difference between Min. and Max.: %0.3f" % min_max_difference)
print("Root Mean Squared Error: %0.3f" % root_mean_squared_error)

calibration_data = pd.DataFrame()
calibration_data["predictions"] = pd.Series(predictions)
calibration_data["targets"] = pd.Series(targets)
calibration_data.describe()

sample = california_housing_dataframe.sample(n=300)

# Get the min and max total_rooms values.
x_0 = sample["total_rooms"].min()
x_1 = sample["total_rooms"].max()

# Retrieve the final weight and bias generated during training.
weight = linear_regressor.get_variable_value('linear/linear_model/total_rooms/weights')[0]
bias = linear_regressor.get_variable_value('linear/linear_model/bias_weights')

# Get the predicted median_house_values for the min and max total_rooms values.
y_0 = weight * x_0 + bias 
y_1 = weight * x_1 + bias

# Plot our regression line from (x_0, y_0) to (x_1, y_1).
plt.plot([x_0, x_1], [y_0, y_1], c='r')

# Label the graph axes.
plt.ylabel("median_house_value")
plt.xlabel("total_rooms")

# Plot a scatter plot from our data sample.
plt.scatter(sample["total_rooms"], sample["median_house_value"])

# Display graph.
plt.show()

def train_model(learning_rate, steps, batch_size, input_feature="total_rooms"):
  """Trains a linear regression model of one feature.
  
  Args:
    learning_rate: A `float`, the learning rate.
    steps: A non-zero `int`, the total number of training steps. A training step
      consists of a forward and backward pass using a single batch.
    batch_size: A non-zero `int`, the batch size.
    input_feature: A `string` specifying a column from `california_housing_dataframe`
      to use as input feature.
  """
  
  periods = 10
  steps_per_period = steps / periods

  my_feature = input_feature
  my_feature_data = california_housing_dataframe[[my_feature]]
  my_label = "median_house_value"
  targets = california_housing_dataframe[my_label]

  # Create feature columns.
  feature_columns = [tf.feature_column.numeric_column(my_feature)]
  
  # Create input functions.
  training_input_fn = lambda:my_input_fn(my_feature_data, targets, batch_size=batch_size)
  prediction_input_fn = lambda: my_input_fn(my_feature_data, targets, num_epochs=1, shuffle=False)
  
  # Create a linear regressor object.
  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=feature_columns,
      optimizer=my_optimizer
  )

  # Set up to plot the state of our model's line each period.
  plt.figure(figsize=(15, 6))
  plt.subplot(1, 2, 1)
  plt.title("Learned Line by Period")
  plt.ylabel(my_label)
  plt.xlabel(my_feature)
  sample = california_housing_dataframe.sample(n=300)
  plt.scatter(sample[my_feature], sample[my_label])
  colors = [cm.coolwarm(x) for x in np.linspace(-1, 1, periods)]

  # Train the model, but do so inside a loop so that we can periodically assess
  # loss metrics.
  print("Training model...")
  print("RMSE (on training data):")
  root_mean_squared_errors = []
  for period in range (0, periods):
    # Train the model, starting from the prior state.
    linear_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period
    )
    # Take a break and compute predictions.
    predictions = linear_regressor.predict(input_fn=prediction_input_fn)
    predictions = np.array([item['predictions'][0] for item in predictions])
    
    # Compute loss.
    root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(predictions, targets))
    # Occasionally print the current loss.
    print("  period %02d : %0.2f" % (period, root_mean_squared_error))
    # Add the loss metrics from this period to our list.
    root_mean_squared_errors.append(root_mean_squared_error)
    # Finally, track the weights and biases over time.
    # Apply some math to ensure that the data and line are plotted neatly.
    y_extents = np.array([0, sample[my_label].max()])
    
    weight = linear_regressor.get_variable_value('linear/linear_model/%s/weights' % input_feature)[0]
    bias = linear_regressor.get_variable_value('linear/linear_model/bias_weights')

    x_extents = (y_extents - bias) / weight
    x_extents = np.maximum(np.minimum(x_extents,
                                      sample[my_feature].max()),
                           sample[my_feature].min())
    y_extents = weight * x_extents + bias
    plt.plot(x_extents, y_extents, color=colors[period]) 
  print("Model training finished.")

  # Output a graph of loss metrics over periods.
  plt.subplot(1, 2, 2)
  plt.ylabel('RMSE')
  plt.xlabel('Periods')
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(root_mean_squared_errors)

  # Output a table with calibration data.
  calibration_data = pd.DataFrame()
  calibration_data["predictions"] = pd.Series(predictions)
  calibration_data["targets"] = pd.Series(targets)
  display.display(calibration_data.describe())

  print("Final RMSE (on training data): %0.2f" % root_mean_squared_error)
    
train_model(
    learning_rate=0.00002,
    steps=1000,
    batch_size=5,
    input_feature="population"
)

ModuleNotFoundError: No module named 'tensorflow'

In [11]:
print("This line will be printed.")


This line will be printed.


In [12]:
x = 1
if x == 1:
    # indented four spaces
    print("x is 1.")

x is 1.


In [13]:
print("Goodbye, World!")

Goodbye, World!


In [14]:
myint = 7
print(myint)

7


In [15]:
myfloat = 7.0
print(myfloat)
myfloat = float(7)
print(myfloat)

7.0
7.0


In [16]:
mystring = 'hello'
print(mystring)
mystring = "hello"
print(mystring)

hello
hello


In [17]:
mystring = "Don't worry about apostrophes"
print(mystring)

Don't worry about apostrophes


In [18]:
one = 1
two = 2
three = one + two
print(three)

hello = "hello"
world = "world"
helloworld = hello + " " + world
print(helloworld)

3
hello world


In [19]:
a, b = 3, 4
print(a,b)

3 4


In [20]:
# This will not work!
one = 1
two = 2
hello = "hello"

print(one + two + hello)

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [24]:
# change this code
mystring = "hello"
myfloat = 30.0
myint = 40

# testing code
if mystring == "hello":
    print("String: %s" % mystring)
if isinstance(myfloat, float) and myfloat == 30.0:
    print("Float: %f" % myfloat)
if isinstance(myint, int) and myint == 40:
    print("Integer: %d" % myint)

String: hello
Float: 30.000000
Integer: 40


In [25]:
mylist = []
mylist.append(1)
mylist.append(2)
mylist.append(3)
print(mylist[0]) # prints 1
print(mylist[1]) # prints 2
print(mylist[2]) # prints 3

# prints out 1,2,3
for x in mylist:
    print(x)

1
2
3
1
2
3


In [26]:
mylist = [1,2,3]
print(mylist[10])

IndexError: list index out of range

In [28]:
numbers = []
strings = []
names = ["Ryan", "Kate", "Nolan"]

# write your code here
numbers.append(1)
numbers.append(2)
numbers.append(3)

strings.append("hello")
strings.append("world")

second_name = names[1]

# this code should write out the filled arrays and the second name in the names list (Kate).
print(numbers)
print(strings)
print("The second name on the names list is %s" % second_name)

[1, 2, 3]
['hello', 'world']
The second name on the names list is Kate


In [29]:
number = 1 + 2 * 3 / 4.0
print(number)

2.5


In [30]:
remainder = 11 % 3
print(remainder)

2


In [31]:
squared = 7 ** 2
cubed = 2 ** 3
print(squared)
print(cubed)

49
8


In [32]:
helloworld = "hello" + " " + "world"
print(helloworld)

hello world


In [33]:
lotsofhellos = "hello" * 10
print(lotsofhellos)

hellohellohellohellohellohellohellohellohellohello


In [34]:
even_numbers = [2,4,6,8]
odd_numbers = [1,3,5,7]
all_numbers = odd_numbers + even_numbers
print(all_numbers)

[1, 3, 5, 7, 2, 4, 6, 8]


In [35]:
print([1,2,3] * 3)

[1, 2, 3, 1, 2, 3, 1, 2, 3]


In [37]:
x = object()
y = object()

# TODO: change this code
x_list = [x] * 20
y_list = [y] * 20
big_list = x_list + y_list

print("x_list contains %d objects" % len(x_list))
print("y_list contains %d objects" % len(y_list))
print("big_list contains %d objects" % len(big_list))

# testing code
if x_list.count(x) == 20 and y_list.count(y) == 20:
    print("Almost there...")
if big_list.count(x) == 20 and big_list.count(y) == 20:
    print("Great!")

x_list contains 20 objects
y_list contains 20 objects
big_list contains 40 objects
Almost there...
Great!


In [38]:
# This prints out "Hello, John!"
name = "John"
print("Hello, %s!" % name)

Hello, John!


In [39]:
# This prints out "John is 23 years old."
name = "John"
age = 23
print("%s is %d years old." % (name, age))

John is 23 years old.


In [40]:
# This prints out: A list: [1, 2, 3]
mylist = [1,2,3]
print("A list: %s" % mylist)

A list: [1, 2, 3]


In [41]:
data = ("John", "Doe", 53.44)
format_string = "Hello %s %s. Your current balance is $%s."

print(format_string % data)

Hello John Doe. Your current balance is $53.44.


In [43]:
astring = "Hello world!"
astring2 = 'Hello world!'

In [44]:
astring = "Hello world!"
print("single quotes are ' '")

print(len(astring))

single quotes are ' '
12


In [45]:
astring = "Hello world!"
print(astring.index("o"))

4


In [46]:
astring = "Hello world!"
print(astring.count("l"))

3


In [47]:
astring = "Hello world!"
print(astring[3:7])

lo w


In [48]:
astring = "Hello world!"
print(astring[3:7:2])

l 


In [49]:
astring = "Hello world!"
print(astring[3:7])
print(astring[3:7:1])

lo w
lo w


In [ ]:
astring = "Hello world!"
print(astring[::-1])

In [ ]:
astring = "Hello world!"
print(astring.upper())
print(astring.lower())

In [52]:
astring = "Hello world!"
print(astring.startswith("Hello"))
print(astring.endswith("asdfasdfasdf"))

True
False


In [53]:
astring = "Hello world!"
afewwords = astring.split(" ")

In [54]:
s = "Strings are awesome!"
# Length should be 20
print("Length of s = %d" % len(s))

# First occurrence of "a" should be at index 8
print("The first occurrence of the letter a = %d" % s.index("a"))

# Number of a's should be 2
print("a occurs %d times" % s.count("a"))

# Slicing the string into bits
print("The first five characters are '%s'" % s[:5]) # Start to 5
print("The next five characters are '%s'" % s[5:10]) # 5 to 10
print("The thirteenth character is '%s'" % s[12]) # Just number 12
print("The characters with odd index are '%s'" %s[1::2]) #(0-based indexing)
print("The last five characters are '%s'" % s[-5:]) # 5th-from-last to end

# Convert everything to uppercase
print("String in uppercase: %s" % s.upper())

# Convert everything to lowercase
print("String in lowercase: %s" % s.lower())

# Check how a string starts
if s.startswith("Str"):
    print("String starts with 'Str'. Good!")

# Check how a string ends
if s.endswith("ome!"):
    print("String ends with 'ome!'. Good!")

# Split the string into three separate strings,
# each containing only a word
print("Split the words of the string: %s" % s.split(" "))

Length of s = 20
The first occurrence of the letter a = 8
a occurs 2 times
The first five characters are 'Strin'
The next five characters are 'gs ar'
The thirteenth character is 'a'
The characters with odd index are 'tig r wsm!'
The last five characters are 'some!'
String in uppercase: STRINGS ARE AWESOME!
String in lowercase: strings are awesome!
String starts with 'Str'. Good!
String ends with 'ome!'. Good!
Split the words of the string: ['Strings', 'are', 'awesome!']


In [55]:
x = 2
print(x == 2) # prints out True
print(x == 3) # prints out False
print(x < 3) # prints out True

True
False
True


In [56]:
name = "John"
age = 23
if name == "John" and age == 23:
    print("Your name is John, and you are also 23 years old.")

if name == "John" or name == "Rick":
    print("Your name is either John or Rick.")

Your name is John, and you are also 23 years old.
Your name is either John or Rick.


In [57]:
name = "John"
if name in ["John", "Rick"]:
    print("Your name is either John or Rick.")

Your name is either John or Rick.


In [58]:
statement = False
another_statement = True
if statement is true:
    # do something
    pass
elif another_statement is true: # else if
    # do something else
    pass
else:
    # do another thing
    pass

NameError: name 'true' is not defined

In [ ]:
x = 2
if x == 2:
    print("x equals two!")
else:
    print("x does not equal to two.")

In [60]:
x = [1,2,3]
y = [1,2,3]
print(x == y) # Prints out True
print(x is y) # Prints out False

True
False


In [61]:
print(not False) # Prints out True
print((not False) == (False)) # Prints out False

True
False


In [ ]:
# change this code
number = 16
second_number = 0
first_array = [1,2,3]
second_array = [1,2]

if number > 15:
    print("1")

if first_array:
    print("2")

if len(second_array) == 2:
    print("3")

if len(first_array) + len(second_array) == 5:
    print("4")

if first_array and first_array[0] == 1:
    print("5")

if not second_number:
    print("6")

In [ ]:
import math 

grades = [100, 90, 70, 30]
weights = [.3,.2,.2,.3]

def GradeCalc (grade, weight):
  
  num = 0.0
  
  for x in range(len (grade)):
    num += grade[x] * weight[x]
    
    
  print (num)
  
GradeCalc(grades, weights)

def StandDev (grade):

  mean = 0.0
  for x in range(len(grade)):
    mean += grade[x]
  mean = mean/len(grade)
  sum = 0.0
  for y in range(len(grade)):
      sum += (mean - y)**0.5
  divided = sum/len(grade)
  stDeviation = (divided)**0.5
  print("Standard Deviation is ", stDeviation)
    
StandDev(grades)
    

In [ ]:
# Author: Karington Watkins 
# Date: 11/21/2019
# Description: Edited CSV lists thingy that takes information that exports the information into organized lists
#               edited / commented for the code from an assignment for a Computer Science Class

#
# Single string containing CSV formatted song data
#  "artist,album,title,duration"
#  Note: Duration is specified in seconds
#
singleSongCSV = "Jimmy Buffett,Songs You Know by Heart,Cheeseburger in Paradise,172"

#
# List of strings containing CSV formatted song data
# within the SongList item that is devided into diferent sections based on where the "," is located 
songList = ['Jimmy Buffett,Songs You Know by Heart,Cheeseburger in Paradise,172', 
            'Jimmy Buffett,Songs You Know by Heart,He Went to Paris,209',
            'Jimmy Buffett,Songs You Know by Heart,Fins,205',
            'Jimmy Buffett,Songs You Know by Heart,Son of a Son of a Sailor,205',
            'Jimmy Buffett,Songs You Know by Heart,A Pirate Looks at Forty,232',
            'Jimmy Buffett,Songs You Know by Heart,Margaritaville,251',
            'Jimmy Buffett,Songs You Know by Heart,Come Monday,189',
            'Jimmy Buffett,Songs You Know by Heart,Changes in Latitudes Changes in Attitudes,195',
            "Jimmy Buffett,Songs You Know by Heart,Why Don't We Get Drunk,162",
            'Jimmy Buffett,Songs You Know by Heart,Pencil Thin Mustache,170',
            'Jimmy Buffett,Songs You Know by Heart,Grapefruit-Juicy Fruit,176',
            'Jimmy Buffett,Songs You Know by Heart,Boat Drinks,157',
            'Jimmy Buffett,Songs You Know by Heart,Volcano,218',
            'Jeffery Epstien,Did Not Kill Himself,Suisided,911',
            'Gucci Gang,Songs n Stuff,Yeet A Score YEET REKT,211',]

testlist =['looping','with','the','list','by','commas','and','epstien','did','not','kill','himself']
# Display nicely formatted song details for 
#   string provided in the following format:
#   "Artist,Album,Title,Duration"
#   testlist was a function to test if the code was going to export the infomation to begin with
# Parameters
#   song - String containing comma separated song details
#
# Return
#   none
def printSong(song):
     #print(type(first song) 
    elementList= song.split(",")#based on the , split then brake it based on new artist, album,title, etc
    print(elementList)
    print(type(elementList))
    artist = elementList[0]
    album = elementList[1]
    title = elementList[2]
    duration = elementList[3]
                                #defines what is exported to text information output in combonation to the list
    output = "\nArtists: " + artist
    output += "\nAlbum: " + album
    output += "\nTitle: " + title
    output += "\nDuration: " + duration
    print(output)

#
# call printSong() to print singleSongCSV
#

printSong(singleSongCSV)

print("\n" + str(len(songList))+ "songs in the list")    

#
# use a for loop and printSong() to print each song in songList
#
#for i in range(0,len(songList)):
 #   printSong(songList[i])
#makes the songList into motion
for song in songList:
           printSong(song)

#for i in testList:
#    print(i)

#  © 2019 GitHub, Inc.
# Terms
#Privacy
#Security


In [ ]:
void CreateRoad()
    {
       
        RoadExit = JustSpawned.transform.Find("ExitPoint");
        SpawnPosition.transform.position = RoadExit.transform.position;
        JustSpawned = Instantiate<GameObject>(ReturnRoad());
        JustSpawned.transform.position = SpawnPosition.transform.position;

        Destroy(Roads[0]);
        Roads[0] = null;
        for (int i = 0; i < (Roads.Length - 1); i++)
        {
            if (Roads[i] == null && Roads[i + 1] != null)
            {
                Roads[i] = Roads[i + 1];
                Roads[i + 1] = null;
            }
        }
        Roads[Roads.Length - 1] = JustSpawned;
    }


This is a script I made to create an infinite road in a driving game. I have another function which returns random road 'pieces' aka 3d models. The basics of what are going on here is that index 0 of the array which holds the road pieces is destroyed, all the road pieces are moved down 1 place in the array, and then the new piece of road is instantiated at the end of the array. Effectively this makes an infinite road that gets deleted over time (as you drive past it). Okay thank you for coming to my TED talk.

In [ ]:
using System.Collections;
using System.Collections.Generic;
using System.Linq;
using UnityEngine;

public class GenerateTerrain : MonoBehaviour {

    [SerializeField]
    private GameObject blockPrefab;//use a unit cube (1x1x1 like unity's default cube)

    [SerializeField]
    private int chunkSize = 50;

    [SerializeField]
    private float noiseScale = .05f;

    [SerializeField, Range(0, 1)]
    private float threshold = .5f;

    [SerializeField]
    private Material material;

    [SerializeField]
    private bool sphere = false;

    private List<Mesh> meshes = new List<Mesh>();//used to avoid memory issues

    private void Generate() {
        float startTime = Time.realtimeSinceStartup;

        #region Create Mesh Data

        List<CombineInstance> blockData = new List<CombineInstance>();//this will contain the data for the final mesh
        MeshFilter blockMesh = Instantiate(blockPrefab, Vector3.zero, Quaternion.identity).GetComponent<MeshFilter>();//create a unit cube and store the mesh from it

        //go through each block position
        for (int x = 0; x < chunkSize; x++) {
            for (int y = 0; y < chunkSize; y++) {
                for (int z = 0; z < chunkSize; z++) {

                    float noiseValue = Perlin3D(x * noiseScale, y * noiseScale, z * noiseScale);//get value of the noise at given x, y, and z.
                    if (noiseValue >= threshold) {//is noise value above the threshold for placing a block?

                        //ignore this block if it's a sphere and it's outside of the radius (ex: in the corner of the chunk, outside of the sphere)
                        //distance between the current point with the center point. if it's larger than the radius, then it's not inside the sphere.
                        float raduis = chunkSize / 2;
                        if (sphere && Vector3.Distance(new Vector3(x, y, z), Vector3.one * raduis) > raduis)
                            continue;

                        blockMesh.transform.position = new Vector3(x, y, z);//move the unit cube to the intended position
                        CombineInstance ci = new CombineInstance {//copy the data off of the unit cube
                            mesh = blockMesh.sharedMesh,
                            transform = blockMesh.transform.localToWorldMatrix,
                        };
                        blockData.Add(ci);//add the data to the list
                    }

                }
            }
        }

        Destroy(blockMesh.gameObject);//original unit cube is no longer needed. we copied all the data we need to the block list.

        #endregion

        #region Separate Mesh Data

        //divide meshes into groups of 65536 vertices. Meshes can only have 65536 vertices so we need to divide them up into multiple block lists.

        List<List<CombineInstance>> blockDataLists = new List<List<CombineInstance>>();//we will store the meshes in a list of lists. each sub-list will contain the data for one mesh. same data as blockData, different format.
        int vertexCount = 0;
        blockDataLists.Add(new List<CombineInstance>());//initial list of mesh data
        for (int i = 0; i < blockData.Count; i++) {//go through each element in the previous list and add it to the new list.
            vertexCount += blockData[i].mesh.vertexCount;//keep track of total vertices
            if (vertexCount > 65536) {//if the list has reached it's capacity. if total vertex count is more then 65536, reset counter and start adding them to a new list.
                vertexCount = 0;
                blockDataLists.Add(new List<CombineInstance>());
                i--;
            } else {//if the list hasn't yet reached it's capacity. safe to add another block data to this list 
                blockDataLists.Last().Add(blockData[i]);//the newest list will always be the last one added
            }
        }

        #endregion

        #region Create Mesh

        //the creation of the final mesh from the data.

        Transform container = new GameObject("Meshys").transform;//create container object
        foreach (List<CombineInstance> data in blockDataLists) {//for each list (of block data) in the list (of other lists)
            GameObject g = new GameObject("Meshy");//create gameobject for the mesh
            g.transform.parent = container;//set parent to the container we just made
            MeshFilter mf = g.AddComponent<MeshFilter>();//add mesh component
            MeshRenderer mr = g.AddComponent<MeshRenderer>();//add mesh renderer component
            mr.material = material;//set material to avoid evil pinkness of missing texture
            mf.mesh.CombineMeshes(data.ToArray());//set mesh to the combination of all of the blocks in the list
            meshes.Add(mf.mesh);//keep track of mesh so we can destroy it when it's no longer needed
            //g.AddComponent<MeshCollider>().sharedMesh = mf.sharedMesh;//setting colliders takes more time. disabled for testing.
        }

        #endregion

        Debug.Log("Loaded in " + (Time.realtimeSinceStartup - startTime) + " Seconds.");
    }

    private void Update() {
        if (Input.GetKeyDown(KeyCode.G)) {
            Destroy(GameObject.Find("Meshys"));//destroy parent gameobject as well as children.
            foreach (Mesh m in meshes)//meshes still exist even though they aren't in the scene anymore. destroy them so they don't take up memory.
                Destroy(m);
            Generate();
        }
    }

    //dunno how this works. copied it from somewhere.
    public static float Perlin3D(float x, float y, float z) {
        float ab = Mathf.PerlinNoise(x, y);
        float bc = Mathf.PerlinNoise(y, z);
        float ac = Mathf.PerlinNoise(x, z);

        float ba = Mathf.PerlinNoise(y, x);
        float cb = Mathf.PerlinNoise(z, y);
        float ca = Mathf.PerlinNoise(z, x);

        float abc = ab + bc + ac + ba + cb + ca;
        return abc / 6f;
    }

}

In [ ]:
source = GetComponent<AudioSource>();                           
source.Stop();               
source.clip = Microphone.Start(null, true, 1, AudioSettings.outputSampleRate);               
source.Play();